<a href="https://colab.research.google.com/github/samanthasu12/Contentstack_Internship/blob/main/TADataCalculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Welcome to my TA Automation Project!**
----

Here are the steps in which you need to take to automate the data.

1. Make sure that all the sheets are completely blank except for the column titles.

2. Insert the Hired AND Viable Candidates report into the Hired AND Viable Candidates sheet. **NOTE:** The report must contain the following columns: First Name,	Last Name,	URM,	Department,	Job Name,	Start Date (Application),	Status,	Offices,	Requisition ID,	Recruiter (the order doesn't matter though).

3. Make sure the sheet names in the google sheet are the following: Sheet1, Hired and Viable Candidates Table, India Only, India Only Table, US/EMEA/ROW Only, US/EMEA/ROW Only Table, and Hired AND Viable Candidates. (Do not change the names of the sheets or any of the column titles!)

4. After you've completed your checks, please click Runtime (located at the very top of the colab) and Run All. It will then ask you to connect the google colab to your own google account, and you just need to sign in.

5. Please give it around 5 seconds to fully automate. (Might be longer depending on how big the report is)

In [ ]:
from google.colab import auth
import pandas as pd
import numpy as np
auth.authenticate_user()
import gspread
from google.auth import default
from oauth2client.client import GoogleCredentials
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import drive
from google.colab import auth
from IPython.display import clear_output
from google.colab import files

creds, _ = default()
drive.mount('/content/drive/')
creds, _ = default()
gc = gspread.authorize(creds)
drive.mount("mnt")

sheet_id='1qZh3oDeFjUqQu8RecKGn-Wb9BE3m0iI3OYab1dBg52I'
#the sheet_id is the "key" to unlocking the spreadsheet that I am importing/exporting onto/from (which in this case is the TA Master Spreadsheet).
#The key is found after /d/ and before /edit in https://docs.google.com/spreadsheets/d/1qZh3oDeFjUqQu8RecKGn-Wb9BE3m0iI3OYab1dBg52I/edit#gid=317505658

sh = gc.open_by_key(sheet_id)
sh.worksheets()
clear_output()

In [ ]:
Hired_And_Viable_Candidates_Worksheet=sh.worksheet('Hired AND Viable Candidates') #This is the Sheet Name
Hired_And_Viable_Candidates_Worksheet=Hired_And_Viable_Candidates_Worksheet.get_all_values()
headers1 = Hired_And_Viable_Candidates_Worksheet.pop(0)
allcandidatesdata=pd.DataFrame(Hired_And_Viable_Candidates_Worksheet, columns=headers1)

The code below reorganizes the data in the format we want it. This organized dataset is stored under the variable frequency_df.

In [ ]:
#I am taking the data from the previous dataset that we just created in the previous cell and automating it here, onto a new dataset.
frequency_df=allcandidatesdata[['Requisition ID', 'Offices','Department','Job Name']].value_counts().reset_index()
frequency_df.columns = ['Requisition ID', 'Offices','Department', 'Job Name', 'Viable Candidates']

countlist=[]
count2list=[]
count3list=[]
count4list=[]
count5list=[]
count6list=[]
extracountlist=[]
extracountlist2=[]

#this chunk of code here is the automation machine; it gets all the numbers in the right places.
for job in frequency_df['Requisition ID']:
  count=0
  count2=0
  count3=0
  count4='N'
  count5=0
  count6=0
  extracount=0
  extracount2='N'
  for idx, row in allcandidatesdata.iterrows():
    if 'Female' in row['URM'] and job in row['Requisition ID']:
      count+=1
    if (job in row['Requisition ID']) and (('Asian / Pacific Islander' in row['URM']) or ('Middle Eastern' in row['URM']) or ('Hispanic / Latinx' in row['URM']) or ('Native Hawaiian / Other Pacific Islander' in row['URM']) or ('African American / Black' in row['URM'])):
      count2+=1
    if (job in row['Requisition ID']) and ('Female' in row['URM']) and (('Asian / Pacific Islander' in row['URM']) or ('Middle Eastern' in row['URM']) or ('Hispanic / Latinx' in row['URM']) or ('Native Hawaiian / Other Pacific Islander' in row['URM']) or ('African American / Black' in row['URM'])):
      count3+=1
      if 'Hired' in row['Status']:
        count4='Y'
        count5+=1
    if job in row['Requisition ID'] and 'Hired' in row['Status']:
      count6+=1
    if (job in row['Requisition ID']) and ('Female' in row['URM']) and 'Hired' in row['Status']:
      extracount+=1
      extracount2='Y'
  countlist.append(count)
  count2list.append(count2)
  count3list.append(count3)
  count4list.append(count4)
  count5list.append(count5)
  count6list.append(count6)
  extracountlist.append(extracount)
  extracountlist2.append(extracount2)
frequency_df['Female Presented']=countlist
frequency_df['URM Presented']=count2list
frequency_df['Total F-URM per job']=count3list
frequency_df['F-URM Hired?']=count4list
frequency_df['Number of F-URM Hired']=count5list
frequency_df['Total Hired']=count6list
frequency_df['Number of Female Hired']=extracountlist
frequency_df['Female Hired?']=extracountlist2

#this chunk of code over here removes all of the interns
for idx, row in frequency_df.iterrows():
  if 'Intern' in row['Job Name']:
    frequency_df.drop([idx], axis=0, inplace=True)

#this sorts it alphabetically, firstly by 'Department' and secondly by 'Job Name'
frequency_df=frequency_df.sort_values(by=['Department', 'Job Name'])
frequency_df=frequency_df.reset_index()
frequency_df=frequency_df.drop('index', axis=1)

#this places the subgroups into their respective main groups
def map_department(value):
    if 'Business Operations' in value or 'People Team' in value or 'General and Administrative' in value or 'Legal' in value or 'Finance' in value:
      return 'G&A'
    if 'Business Development' in value or 'Marketing' in value or 'Product Marketing' in value or 'Revenue Marketing' in value or 'Corporate Communications' in value or 'Digital Experience' in value or 'Operations' in value:
      return 'Marketing'
    if 'TSO' in value or 'Sale Engineering' in value or 'Customer Success' in value or 'Partnerships' in value or 'Growth' in value or 'Partnerships' in value or 'Sales General' in value:
      return 'Sales'
    if 'Customer Support' in value or 'Engineering' in value or 'Product Development' in value or 'Product Management' in value or 'Admin' in value or 'Cloud SRE' in value or 'Cloud - COGS' in value or 'Cloud - R&D' in value or 'Design' in value or 'DevOps' in value:
      return 'Product Dev.'
    else:
        return value

frequency_df['Department'] = frequency_df['Department'].map(map_department)

Run the code below to update the data onto sheet1.

In [ ]:
# Update the 'Sheet1' worksheet with the updated values from 'frequency_df'
update_values = frequency_df.sort_values(by='Department').values.tolist()
Sheet1_Worksheet = sh.worksheet("Sheet1")
Sheet1_Worksheet.update([frequency_df.columns.values.tolist()] + update_values)

# Clear output
clear_output()

Perfect! We've uploaded our first sheet! Let's keep going.


**Now let's automate the table for the previous dataset we just created onto a separate spreadsheet!**

Note: The chart will automate on its own once we import this data.

The code below finds specific values for each **department**.

In [ ]:
departmental_values = frequency_df['Department'].unique()
departmental_values=pd.DataFrame(departmental_values, columns=['Department'])

count7list=[]
count8list=[]
count9list=[]
count10list=[]
count11list=[]
count12list=[]
for value in departmental_values['Department']:
  count7=0
  count8=0
  count9=0
  count10=0
  count11=0
  count12=0
  for idx, row in frequency_df.iterrows():
    if value in row['Department']:
        count7+= row['Viable Candidates']
        count8+= row['Female Presented']
        count9+= row['URM Presented']
        count10+= row['Total F-URM per job']
        count11+= row['Number of F-URM Hired']
  count7list.append(count7)
  count8list.append(count8)
  count9list.append(count9)
  count10list.append(count10)
  count11list.append(count11)
departmental_values['Viable Candidates']=count7list
departmental_values['Total Female Presented']=count8list
departmental_values['Total URM Presented']=count9list
departmental_values['Total F-URM Presented']=count10list
departmental_values['Total F-URM Hired']=count11list
departmental_values['Proportion of F-URM Candidates Provided']=[round(a / b,2) if b != 0 else 0.0 for a, b in zip(count10list, count7list)]
departmental_values['Proportion of F-URM Candidates Hired']=[round(a / b,2) if b != 0 else 0.0 for a, b in zip(count11list, count8list)]

departmental_values=departmental_values.sort_values(by=['Department'])
departmental_values=departmental_values.reset_index()
departmental_values=departmental_values.drop('index', axis=1)

In [ ]:
update_values = departmental_values.sort_values(by='Department').values.tolist()
Sheet1Table_Worksheet = sh.worksheet("Hired and Viable Candidates Table")
Sheet1Table_Worksheet.update([departmental_values.columns.values.tolist()] + update_values)
clear_output()

**Part 4**: Creating a Dataset That Only Includes India-based candidates


---


This creates the dataset for only India-based candidates.

In [ ]:
#this part includes only the rows that include the word India
filtered_dfIndia= pd.DataFrame()
for index, row in frequency_df.iterrows():
    if 'India'  in row['Offices']:
        filtered_dfIndia = filtered_dfIndia.append(row)
filtered_dfIndia=filtered_dfIndia.reset_index()
clear_output()

#this part gets rid of URM and F-URM columns
filtered_dfIndia=filtered_dfIndia.drop('URM Presented', axis=1)
filtered_dfIndia=filtered_dfIndia.drop('Total F-URM per job', axis=1)
filtered_dfIndia=filtered_dfIndia.drop('Number of F-URM Hired', axis=1)
filtered_dfIndia=filtered_dfIndia.drop('F-URM Hired?', axis=1)
filtered_dfIndia=filtered_dfIndia.drop('index', axis=1)


The code below updates the data

In [ ]:
update_values = filtered_dfIndia.sort_values(by='Department').values.tolist()
IndiaWorksheet = sh.worksheet("India Only")
IndiaWorksheet.update([filtered_dfIndia.columns.values.tolist()] + update_values)

{'spreadsheetId': '1qZh3oDeFjUqQu8RecKGn-Wb9BE3m0iI3OYab1dBg52I',
 'updatedRange': "'India Only'!A1:I32",
 'updatedRows': 32,
 'updatedColumns': 9,
 'updatedCells': 288}

In [ ]:
filtered_dfIndia_Table = filtered_dfIndia['Department'].unique()
filtered_dfIndia_Table=pd.DataFrame(filtered_dfIndia_Table, columns=['Department'])

count7list=[]
count8list=[]
count9list=[]
for value in filtered_dfIndia_Table['Department']:
  count7=0
  count8=0
  count9=0
  for idx, row in filtered_dfIndia.iterrows():
    if value in row['Department']:
        count7+= row['Viable Candidates']
        count8+= row['Female Presented']
        count9+= row['Number of Female Hired']
  count7list.append(count7)
  count8list.append(count8)
  count9list.append(count9)
filtered_dfIndia_Table['Viable Candidates']=count7list
filtered_dfIndia_Table['Total Female Presented']=count8list
filtered_dfIndia_Table['Total Female Hired']=count9list
filtered_dfIndia_Table['Proportion of Female Candidates Provided']=[round(a / b,2) if b != 0 else 0.0 for a, b in zip(count8list, count7list)]
filtered_dfIndia_Table['Proportion of Female Candidates Hired']=[round(a / b,2) if b != 0 else 0.0 for a, b in zip(count9list, count8list)]
filtered_dfIndia_Table['Proportion of Proportion of F-URM Candidates Provided']=[round(a / b,2) if b != 0 else 0.0 for a, b in zip(count9list, count8list)]

The code below selects the sheet that I want to import onto

In [ ]:
update_values = filtered_dfIndia_Table.sort_values(by='Department').values.tolist()
IndiaTableWorksheet = sh.worksheet("India Only Table")
IndiaTableWorksheet.update([filtered_dfIndia_Table.columns.values.tolist()] + update_values)
clear_output()

The code below updates the sheet

**Part 4**: Creating a Dataset That Only Includes US/EMEA/ROW-Based Candidates

---

(EXLUDES INDIA!)

In [ ]:
filtered_dfnotIndia = pd.DataFrame()
for index, row in frequency_df.iterrows():
    if 'India' not in row['Offices']:
        filtered_dfnotIndia = filtered_dfnotIndia.append(row)
filtered_dfnotIndia=filtered_dfnotIndia.reset_index()
clear_output()
filtered_dfnotIndia=filtered_dfnotIndia.drop('index', axis=1)

In [ ]:
update_values = filtered_dfnotIndia.sort_values(by='Department').values.tolist()
NotIndiaWorksheet = sh.worksheet("US/EMEA/ROW Only")
NotIndiaWorksheet.update([filtered_dfnotIndia.columns.values.tolist()] + update_values)

{'spreadsheetId': '1qZh3oDeFjUqQu8RecKGn-Wb9BE3m0iI3OYab1dBg52I',
 'updatedRange': "'US/EMEA/ROW Only'!A1:M20",
 'updatedRows': 20,
 'updatedColumns': 13,
 'updatedCells': 260}

In [ ]:
unique_values = filtered_dfnotIndia['Department'].unique()
unique_df=pd.DataFrame(unique_values, columns=['Department'])

count7list=[]
count8list=[]
count9list=[]
count10list=[]
count11list=[]
count12list=[]
for value in unique_df['Department']:
  count7=0
  count8=0
  count9=0
  count10=0
  count11=0
  count12=0
  for idx, row in filtered_dfnotIndia.iterrows():
    if value in row['Department']:
        count7+= row['Viable Candidates']
        count8+= row['Female Presented']
        count9+= row['URM Presented']
        count10+= row['Total F-URM per job']
        count11+= row['Number of F-URM Hired']
  count7list.append(count7)
  count8list.append(count8)
  count9list.append(count9)
  count10list.append(count10)
  count11list.append(count11)
unique_df['Viable Candidates']=count7list
unique_df['Total Female Presented']=count8list
unique_df['Total URM Presented']=count9list
unique_df['Total F-URM Presented']=count10list
unique_df['Number of F-URM Hired']=count11list
unique_df['Proportion of F-URM Candidates Provided']=[round(a / b,2) if b != 0 else 0.0 for a, b in zip(count10list, count7list)]
unique_df['Proportion of F-URM Candidates Hired']=[round(a / b,2) if b != 0 else 0.0 for a, b in zip(count11list, count8list)]

In [ ]:
update_values = unique_df.sort_values(by='Department').values.tolist()
NotIndiaTableWorksheet = sh.worksheet("US/EMEA/ROW Only Table")
NotIndiaTableWorksheet.update([unique_df.columns.values.tolist()] + update_values)

{'spreadsheetId': '1qZh3oDeFjUqQu8RecKGn-Wb9BE3m0iI3OYab1dBg52I',
 'updatedRange': "'US/EMEA/ROW Only Table'!A1:H5",
 'updatedRows': 5,
 'updatedColumns': 8,
 'updatedCells': 40}